# Reinforcement Learning model for forecasting spot prices

###  Libaries & Functions

In [1]:
import sys
import numpy as np
import random
from collections import deque
import numpy as np
import math
import pandas as pd
from datetime import datetime, time
import warnings

import keras
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense
from keras.optimizers import Adam

#Jupyter visualization
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib

#source: https://github.com/edwardhdlu/q-trader 

C:\Users\larsr\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class Agent:
    def __init__(self, state_size, is_eval=False, model_name=""):
        self.state_size = state_size # normalized previous days
        self.action_size = 3 # sit, buy, sell
        self.memory = deque(maxlen=1000)
        self.inventory = []
        self.model_name = model_name
        self.is_eval = is_eval
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        if not is_eval:
            self.model=self._model()
        else:
            self.model=load_seq_model(model_name)

    def _model(self):
        model = Sequential()
        model.add(Dense(units=64, input_shape=(11,5), activation="relu"))
        model.add(keras.layers.Flatten(data_format=None))
        model.add(Dense(units=32, activation="relu"))
        model.add(Dense(units=8, activation="relu"))
        model.add(Dense(self.action_size, activation="linear"))
        model.compile(loss="mse", optimizer=Adam(lr=0.001))
        return model
    def act(self, state):
        if not self.is_eval and random.random() <= self.epsilon:
            randAct=random.randrange(self.action_size)
#             if randAct!=0:
#                 print("ran dom action ",randAct)
            return randAct
        options = self.model.predict(state)
#         print("predicted by the model",np.argmax(options[0]),options)
        return np.argmax(options[0])

    def expReplay(self, batch_size):
        mini_batch = []
        l = len(self.memory)
        for i in range(l - batch_size + 1, l):
            mini_batch.append(self.memory[i])

        for state, action, reward, next_state, done in mini_batch:
            target = reward
#             print('reward',target)
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
#             print(target_f)
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay 

# prints formatted price
def formatPrice(n):
    return ("-€" if n < 0 else "€") + "{0:.2f}".format(abs(n))

# returns the sigmoid
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sigmoid_m(x):
    return 1 / (1 + np.exp(-1.0 * x))

# returns an an n-day state representation ending at time t
def getState(data, t, n):
    d = t - n + 1
    block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # pad with t0
    res = []
    for i in range(n - 1):
        res.append(sigmoid_m(block[i + 1] - block[i]))
    return np.array([res])

def getState_m(data, t, n):
    d = t - n + 1
    if d>= 0:
        block = data[d:t + 1]
    else:
        block = np.repeat(data[0:1], n, axis=0)
        block[n-t:] = data[1:n+d]
    res = []
    for i in range(n - 1):
        res.append(sigmoid_m(block[i + 1] - block[i]))
#     print("shape of array",np.array([res])[0].shape)
    return np.array([res])

def normalize(dataframe,ignore, scalarfile=None, save=True):
    table=dataframe[:]
    normalizer = MinMaxScaler()
    cols_normalize=list(set(dataframe.keys())-set(ignore))
    cols_renamed = [str(col)+ "_n" for col in cols_normalize]
    table[cols_renamed] = table[cols_normalize]
    if save:
        normalizer.fit(table[cols_renamed])
    else:
        normalizer = joblib.load("scalars/"+scalarfile+".save") 
    norm_df = pd.DataFrame(normalizer.transform(table[cols_renamed]), 
                                 columns=cols_renamed, 
                                 index=table.index)
    join_df = table[table.columns.difference(cols_renamed)].join(norm_df)
    table = join_df.reindex(columns = table.columns)
    if save:
        joblib.dump(normalizer, "scalars/"+scalarfile+".save")
    return table, scalarfile, cols_renamed

def save_model(model, filename):
    # serialize model to JSON
    model_json = model.to_json()
    with open("models/"+filename+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("models/"+filename+".h5")
    print("Saved model to disk")

def load_seq_model(filename):
    # load json and create model
    json_file = open("models/"+filename+".json", 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("models/"+filename+".h5")
    print('`load_seq_model` > model and weights loaded : ', filename)
    return loaded_model

###  Run model

In [3]:
#source APX spot prices: https://transparency.entsoe.eu/dashboard/show
#source generation/consumption: https://data.open-power-system-data.org/time_series/

key = 'Day_Ahead_Prices_NL'
data = pd.read_csv("data/" + key + ".csv", delimiter=';')
data['PRICE']=pd.to_numeric(data['PRICE'], errors='coerce')
data['PRICE']= data['PRICE'].interpolate()
data['WIND_OFFSHORE']= data['WIND_OFFSHORE'].interpolate()
data['WIND_ONSHORE']= data['WIND_ONSHORE'].interpolate()
data_train = data[100000:104001] # 1-1-2016 <> 12-11-2017 
data_test = data[104001:108817] # 12-12-2017 <> 31-05-2018 
data_train_n, scalarfile, cols_n = normalize(data_train, ['TIMESTAMP', 'DATE', 'LOAD_1'],"MMS_"+datetime.today().strftime('%Y%m%d'), save=True)
data_test_n, scalarfile, cols_n = normalize(data_test, ['TIMESTAMP', 'DATE', 'LOAD_1'],"MMS_20190125", save=False)
#data = list(data['PRICE']) #OLD 1D VERSION


In [5]:
from keras import backend as K

def run_model(data, window_size, episode_count, batch_size, logtype=0, training=True, model_name=None):
    K.clear_session()

    data_m = np.array(np.asmatrix(data[['PRICE_n', 'LOAD_2_n', 'SOLAR', 'WIND_ONSHORE', 'WIND_OFFSHORE']]))
    is_eval = not training
    agent = Agent(window_size, is_eval, model_name=model_name) 
    l = len(data_m) - 1
    total_profits=[]
    for e in range(episode_count):
        print("Episode " + str(e+1) + "/" + str(episode_count))
        state = getState_m(data_m, 0, window_size + 1)
        total_profit = 0
        agent.inventory = []
        for t in range(l):
            if t%100==0:
                print(t,l)
            action = agent.act(state)
            next_state = getState_m(data_m, t + 1, window_size + 1)
            reward = 0
            if action == 1: # buy
                agent.inventory.append(data['PRICE'].iloc[t])
                if logtype==2:
                    print("Buy: " + formatPrice(data['PRICE'].iloc[t]))
            elif action == 2 and len(agent.inventory) > 0: # sell
                bought_price = agent.inventory.pop(0)
                reward = max(data['PRICE'].iloc[t] - bought_price, 0)
                total_profit += data['PRICE'].iloc[t] - bought_price
                if logtype>=2:
                    print("Sell: " + formatPrice(data['PRICE'].iloc[t]) + " | Profit: " + formatPrice(data['PRICE'].iloc[t] - bought_price))
            done = True if t == l - 1 else False
            agent.memory.append((state, action, reward, next_state, done))

            state = next_state

            if training and (len(agent.memory) > batch_size):
                agent.expReplay(batch_size)
            if done:
                if logtype>=1:
                    print("--------------------------------")
                    print("Total Profit: " + formatPrice(total_profit))
                    print("--------------------------------")
                total_profits.append(total_profit)
    if training:
        save_model(agent.model, "RL_"+datetime.today().strftime('%Y%m%d'))
    print('Total profit after ', episode_count, ' episodes: ', np.sum(total_profits)/episode_count)
    return total_profits

total_profits =  run_model(data_train_n, 11, 10, 32, logtype=0, training=True)
newDf=pd.DataFrame(total_profits)
newDf.to_csv("total profits ")
#logtype 0: log end result, 1: log intermediate result, 2: log actions
# total_profits = run_model(data_train_n, 11, 1, 64, logtype=0, training=False, model_name='RL_20190207')

Episode 1/10
0 4000
100 4000
200 4000
300 4000


KeyboardInterrupt: 

###  Vizualize

In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.style.use('seaborn-whitegrid')

# fig = plt.figure(figsize=(8, 6), dpi=80)
# ax = plt.axes()
# ax2 = ax.twinx() 

# t_min = 0
# t_max = 1000

# data_v = data_train

# ax.plot(data_v.index[t_min:t_max], data_v.PRICE[t_min:t_max], color='blue')
# ax2.plot(data_v.index[t_min:t_max], data_v.LOAD_2[t_min:t_max], color='grey')
# ax2.plot(data_v.index[t_min:t_max], data_v.SOLAR[t_min:t_max], color='yellow')
# ax2.plot(data_v.index[t_min:t_max], data_v.WIND_ONSHORE[t_min:t_max], color='red')
# ax2.plot(data_v.index[t_min:t_max], data_v.WIND_OFFSHORE[t_min:t_max], color='#FFAA55')